In [1]:
import pygame
import numpy as np


pygame 2.0.1 (SDL 2.0.14, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [1]:
import pygame
import numpy as np
pygame.init()

largura, altura = 800, 800
WIN = pygame.display.set_mode((largura, altura))
pygame.display.set_caption("Sistema Solar Simulado")
BRANCO = (255,255,255)
AMARELO = (255,200,0)
AZUL = (100,149,237)
AZUL_CLARO = (173,216,230)
VERMELHO = (187,39,50)
CINZA_ESCURO = (80,78,81)

FONT = pygame.font.SysFont("comicsans", 20)

class Planeta:
    AU = 149.6e6 * 1000
    G = 6.67428e-11
    SCALE = 250/AU
    TIMESTEP = 3600*24  #Um dia do planeta
    
    def __init__(self,x,y,raio,cor, massa):
        self.x = x
        self.y = y
        self.raio = raio
        self.cor = cor
        self.massa = massa
        self.orbita=[]
        self.sun = False
        self.distance_ate_sol = 0
        
        self.x_v = 0
        self.y_v = 0
    
    def desenhar(self,win):
        x = self.x * self.SCALE + largura/2
        y = self.y * self.SCALE + altura/2
        
        if len(self.orbita) >= 2:
            atualizar_pontos = []
            for ponto in self.orbita:
                x_p, y_p = ponto
                x_p = x_p*self.SCALE + largura/2
                y_p = y_p*self.SCALE + altura/2
                atualizar_pontos.append((x_p, y_p))
            pygame.draw.lines(win,self.cor, False, atualizar_pontos, 2)  
        
        pygame.draw.circle(win, self.cor,(x,y), self.raio)
        
        if not self.sun:
            distancia_texto = FONT.render(f"{round(self.distance_ate_sol/1000, 1)}km",1 , BRANCO)
            win.blit(distancia_texto, (x - distancia_texto.get_width()/2, y - distancia_texto.get_height()/2))
        
        
    def atracao(self,outro):
        outro_x, outro_y = outro.x, outro.y
        distancia_x = outro_x - self.x
        distancia_y = outro_y - self.y
        distancia =np.sqrt( distancia_x**2 + distancia_y**2)
        
        if outro.sun:
            self.distance_ate_sol = distancia
            
        forca = self.G * self.massa * outro.massa / (distancia**2)
        
        theta = np.arctan2(distancia_y,distancia_x)
        
        forca_x = forca * np.cos(theta)
        forca_y = forca * np.sin(theta)
        
        return forca_x, forca_y
    
    def atualiza_posicao(self, planetas):
        total_fx = total_fy = 0
        for planeta in planetas:
            if self == planeta:
                continue
            fx, fy = self.atracao(planeta)
            total_fx += fx
            total_fy += fy
        
        self.x_v += (total_fx/self.massa) * self.TIMESTEP
        self.y_v += (total_fy/self.massa) * self.TIMESTEP
        
        self.x += self.x_v * self.TIMESTEP
        self.y += self.y_v * self.TIMESTEP
        if len(self.orbita) <= 4*180:
            self.orbita.append((self.x,self.y))
        

        
def main():
    run =True
    clock = pygame.time.Clock()
    
    sol = Planeta(0,0,30, AMARELO, 1.98892* 10**30)
    sol.sun = True
    
    terra = Planeta(-1*Planeta.AU,0,16, AZUL, 5.9742* 10**24)
    terra.y_v = 29.783*1000
    
    marte = Planeta(-1.524*Planeta.AU,0,12, VERMELHO, 6.39* 10**23)
    marte.y_v = 24.077 *1000
    
    mercurio = Planeta(0.387*Planeta.AU,0,8, CINZA_ESCURO, 3.30*(10**23))
    mercurio.y_v = -47.4 *1000
    
    venus = Planeta(0.723*Planeta.AU,0,14, AZUL_CLARO, 4.8685* 10**24)
    venus.y_v = -35.02 *1000
    
    planetas = [sol,terra,marte,mercurio,venus]
    while run:
        clock.tick(60)
        WIN.fill((0,0,0))
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                
        for planeta in planetas:
            planeta.atualiza_posicao(planetas)
            planeta.desenhar(WIN)
            
        pygame.display.update()
    pygame.quit()

main()

pygame 2.0.1 (SDL 2.0.14, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html
